In [9]:
#Import everything 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import csv

import matplotlib.pyplot as plt
plt.style.use('seaborn')


In [ ]:
#starting sentiment analysis 

import nltk
from nltk.corpus import stopwords

#Lemmatization
#Basically changing word like resturants to resturant(words to word) bc really they mean the same thing 

import textblob

from textblob import Word

from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

#Sentiment Analysis

from textblob import TextBlob

In [ ]:
#Create a function to gather average number of words per review

def average_words(x):
    words = x.split() #to give the # of words
    return sum(len(word)for word in words) / len(words)

In [ ]:
movie_df = pd.read_csv('/Users/markdetiberiis/MovieMonths/movie_df.csv')

In [10]:
f = open('/Users/markdetiberiis/MovieMonths/movie_titles.csv')
csv_f = csv.reader(f)

In [11]:
with open('/Users/markdetiberiis/MovieMonths/movie_titles.csv', newline='') as f:
    reader = csv.reader(f)
    titles = list(reader)



In [12]:
new_list = []
for v in titles:
    new_list.append(v[0])

In [13]:
new_list[10]

'The_Rum_Diary'

In [ ]:
#stop words that dont add value

other_stopwords = ['us','get','also','would','end']

In [87]:
#Need a counter to count which index to set primary_title equal too
#Need a sleep counter to make sure webpage dosent block us 

for title in new_list:
    R = requests.get('https://www.rottentomatoes.com/m/{}'.format(title))
    if R.status_code == 404:
        counter += 1
        continue 
    elif R.status_code == 500:
        counter += 1
        continue 
    else:
        soup = BeautifulSoup(R.text, 'html.parser')
#collect the div content 
        divs = soup.findAll(class_='media-body quote_bubble__quote')

        reviews = []
        for div in divs:
            reviews.append(div.find('p').text)

        clean_reviews = []

        for x in reviews:
            clean_reviews.append(x.strip())

        reviews_df = pd.DataFrame(np.array(clean_reviews), columns = ['review'])

#add movie name 
        reviews_df['primary_title'] = title

        #reviews_df['movie_name'] = new_list[title]

#Gather the word length of each review

        reviews_df['word_count'] = reviews_df['review'].apply(lambda x: len(x.split()))

#Gather the character count of each review

        reviews_df['char_count'] = reviews_df['review'].apply(lambda x: len(x))
        reviews_df['average_word_length'] = reviews_df['review'].apply(lambda x: average_words(x))
        stop_words = stopwords.words('english')

#How many stop words in each review 

        reviews_df['stopword_count'] = reviews_df['review'].apply(lambda x: len([word for word in x.split() if word in stop_words]))
        reviews_df['stopword_percentage'] = reviews_df['stopword_count']/reviews_df['word_count']



#Data Cleaning 

#looping through each word which is x
#splititng x givng us every word
#every word we get back were lowering

        reviews_df['lowercase'] = reviews_df['review'].apply(lambda x: " ".join(word.lower() for word in x.split()))

#removing puncuations
        reviews_df['punctuation'] = reviews_df['lowercase'].astype(str).str.replace('[^\w\s]', '')
        reviews_df['stopwords'] = reviews_df['punctuation'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))
        reviews_df['clean_review'] = reviews_df['stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stopwords))


#an array of every single word in all the reviews

#pd.Series(" ".join(reviews_df['clean_review']).split()).value_counts()[:30]
        reviews_df['lemmatized'] = reviews_df['clean_review'].apply(lambda x: " ".join(Word(word).lemmatize() for word in x.split()))
#Sentiment Analysis
        reviews_df['polarity'] = reviews_df['lemmatized'].apply(lambda x: TextBlob(x).sentiment[0])
        reviews_df['subjectivity'] = reviews_df['lemmatized'].apply(lambda x: TextBlob(x).sentiment[1])
#clean up columns we dont need now
        reviews_df.drop(['lowercase','punctuation','stopwords','clean_review', 'lemmatized'], axis = 1, inplace = True)
        reviews_df['total_polarity'] = reviews_df.polarity.mean()
    
        counter += 1
    count = counter -1
    reviews_df['primary_title'] = movie_df['primary_title'][counter]
#print the df
movie_df.head()

KeyboardInterrupt: 

In [86]:
#reviews_df['polarity'].mean()

movie_df.head()

,Unnamed: 0,Unnamed: 0.1,tconst,primary_title,runtime_minutes,genres,averagerating,numvotes,id,production_budget,domestic_gross,worldwide_gross,release_month,release_day,release_year,total_polarity
0,0,0,tt0249516,Foodfight,91.0,"Action,Animation,Comedy",1.9,8248,26,45000000,0,73706,Dec,31,2012,-0.156318
1,1,1,tt0326592,The_Overnight,88.0,NaN,7.5,24,21,200000,1109808,1165996,Jun,19,2015,-0.156318
2,2,2,tt3844362,The_Overnight,79.0,"Comedy,Mystery",6.1,14828,21,200000,1109808,1165996,Jun,19,2015,-0.156318
3,3,3,tt0337692,On_the_Road,124.0,"Adventure,Drama,Romance",6.1,37886,17,25000000,720828,9313302,Mar,22,2013,-0.156318
4,4,4,tt4339118,On_the_Road,89.0,Drama,6.0,6,17,25000000,720828,9313302,Mar,22,2013,-0.156318
